In [ ]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

# Software Optimizations -- how to write cache friendly code?

## Revisiting the design of your data structures.

### How much space does the following data structures need in physical memory?

In [ ]:
compare([do_render_code("./structure/memory_usage.c", show=["//START_1","//END_1"]),do_render_code("structure/memory_usage.c", show="main")])

In [ ]:
! cd structure; make clean; make memory_usage_A; ./memory_usage_A

Now, let's rearrange the data structure a little bit and see what's going on!

In [ ]:
compare([do_render_code("./structure/memory_usage.c", show=["//START_1","//END_1"]),do_render_code("structure/memory_usage.c", show=["START_2","END_2"])])
! cd structure; make memory_usage_B; ./memory_usage_B

### What's a better data structure?

In [ ]:
compare([do_render_code("./structure/array_of_objects.c", show=["//START","//END"]),do_render_code("structure/object_of_arrays.c", show=["//START","//END"])])

If the main workload is typically similar to 
```SELECT AVG(assignment_1) FROM table ```

In [ ]:
compare([do_render_code("./structure/array_of_objects.c", show=["//START_SELECT","//END_SELECT"]),do_render_code("structure/object_of_arrays.c", show=["//START_SELECT","//END_SELECT"])])

Which one is better?

In [ ]:
! cd structure; make array_of_objects; make object_of_arrays

In [ ]:
! cd structure; echo "array of objects"; time ./array_of_objects 28800 10000 0; echo "object of arrays"; time ./object_of_arrays 28800 10000 0

## Loop fusion/fission/interchance

In a very early lecture, we've learned the order of traversing loop matters a lot! It's actually the very first optimization in making your code cache friendly --

### Loop Interchange

In [ ]:
compare([do_render_code("madd/madd_A.c",show=["//START","//END"]),do_render_code("madd/madd_B.c",show=["//START","//END"])])

Do you remember the code that let Jetson Nano suffer? What if we change it to the right hand side?

### Loop Fission

In [ ]:
compare([do_render_code("loop/loop.c",show=["#else","#endif"]),do_render_code("loop/loop.c",show=["#ifdef A","#else"])])

In [ ]:
! ssh htseng@nano "cd courses/CS203/demo/memory/loop; make clean; make; valgrind --tool=cachegrind ./loop_B 524288 >& nano_B.perf;  grep 'D   refs\|D1' nano_B.perf; valgrind --tool=cachegrind ./loop_A 524288 >& nano_A.perf ; grep 'D   refs\|D1' nano_A.perf"

What if we run it on an intel processor?

In [ ]:
! ssh htseng@azelf "cd courses/CS203/demo/memory/loop; make clean; make; valgrind --tool=cachegrind ./loop_B 524288 >& intel_B.perf;  grep 'D   refs\|D1' intel_B.perf; valgrind --tool=cachegrind ./loop_A 524288 >& intel_A.perf ; grep 'D   refs\|D1' intel_A.perf"

## Case study: matrix multiplications

GEMM that computes C = A $\times$ B is the core of many AI/ML applications. The most naive implementation of GEMM takes $O(n^3)$. Assume it takes 1 second to perform GEMM on 1,024$\times$1,024$\times$1,024 matrices. How much time do you expect it would take for 2,048$\times$2,048$\times$2,048 matrices?

In [ ]:
render_code("matrix_mul/mm.c", show=["//START","//END"])

In [ ]:
! cd matrix_mul; make clean; make mm

In [ ]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > mm.csv
! ./matrix_mul/mm 32 >> ./matrix_mul/mm.csv ;./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv
#! cs203 job memory "./matrix_mul/mm 1024 >> ./matrix_mul/mm.csv ; ./matrix_mul/mm 2048 >> ./matrix_mul/mm.csv"

In [ ]:
display_df_mono(render_csv("matrix_mul/mm.csv"))

WOW! Compuational complexty breaks again! The GEMM performance go wild because of cache misses!

### Matrix tiling algorithm

Let's try to partition GEMM into smaller tiles!

In [ ]:
render_code("matrix_mul/blockmm.c", show=["//START","//END"])

In [ ]:
! cd matrix_mul/; make blockmm

In [ ]:
! cd matrix_mul; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm.csv
! ./matrix_mul/blockmm 32 1 >> ./matrix_mul/blockmm.csv ;./matrix_mul/blockmm 1024 256 >> ./matrix_mul/blockmm.csv ; ./matrix_mul/blockmm 2048 256 >> ./matrix_mul/blockmm.csv

In [ ]:
display_df_mono(render_csv("matrix_mul/blockmm.csv"))

In [ ]:
render_code("matrix_mul/blockmm_transpose.c", show=["//START","//END"])

### Matrix transpose

In [ ]:
! cd matrix_mul; make blockmm_transpose; echo "IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm_transpose.csv
! ./matrix_mul/blockmm_transpose 32 1 >> ./matrix_mul/blockmm_transpose.csv ;./matrix_mul/blockmm_transpose 1024 256 >> ./matrix_mul/blockmm_transpose.csv ; ./matrix_mul/blockmm_transpose 2048 256 >> ./matrix_mul/blockmm_transpose.csv

In [ ]:
display_df_mono(render_csv("matrix_mul/blockmm_transpose.csv"))